Подготовка базы данных

In [1]:
import pandas as pd
import threading
import sys

In [2]:
data = pd.read_csv('Raw_data.csv')

In [3]:
data.shape

(371583, 21)

In [4]:
data = data.query("businessman_flag == 0")
data = data.drop(['term','businessman_flag'], axis=1)

In [5]:
data = data.fillna({'start_date': -1, 'fact_close_date': -1, 'card_type_name':-1})

In [6]:
data = data.astype({'start_date': int, 'fact_close_date': int , 'purchase_sum':int ,'purchase_sum':int ,'contract_sum':int , 'current_balance_avg_sum':int , 'current_balance_sum':int, 'current_debit_turn_sum':int, 'current_credit_turn_sum':int})

In [7]:
data.shape

(371583, 19)

In [8]:
data = data.drop_duplicates() # Избавление от дубликатов записей

Поиск валидных записей

In [9]:
listUniqueCardID = data.card_id.unique()
len(listUniqueCardID)

22583

In [10]:
def getlistEqualPartsOfNumberParts(intNub, intParts):
  d, r = divmod(intNub, intParts)
  return [d + (1 if i < r else 0) for i in range(intParts)]

In [11]:
def th_get_ActualUserData(gl_DataFrame, listUniqueCardID, intIndex):
    local_UsersData = pd.DataFrame()
    for i in range(len(listUniqueCardID)):
        if float(i%100) == 0:
            print(i)
            
        strPDCall = f"card_id == '{listUniqueCardID[i]}'" # Запрос

        #f_Data = data.query(strPDCall)
        #print( listUniqueCardID[i] , '-', f_Data.shape)

        local_Data = data.query(strPDCall).drop_duplicates()
        #print( listUniqueCardID[i] , '---', local_Data.shape,'\n')

        if (local_Data.shape[0] != 1):
              
            maxDebet =  local_Data['current_debit_turn_sum'].max()
            maxCredet = local_Data['current_credit_turn_sum'].max()
            maxPurchaseCount = local_Data['purchase_count'].max()
            
            thisDB = local_Data.query(f"current_debit_turn_sum == {maxDebet} & current_credit_turn_sum == {maxCredet} & purchase_count == {maxPurchaseCount}")
            
        else:
            thisDB = local_Data
        
        #print(i, "-", thisDB.shape, "-", listUniqueCardID[i],'\n')

        local_UsersData = pd.concat([local_UsersData, thisDB]) 
           
        #print(local_UsersData.shape)

    local_UsersData.to_csv(f'New_CSV_{intIndex}.csv')

    gl_DataFrame = pd.concat([gl_DataFrame, local_UsersData]) 
    print(gl_DataFrame.shape)

    print(f'Source_files/JSON/Thread {intIndex} is done !')
    sys.exit

In [12]:
listParts = getlistEqualPartsOfNumberParts(len(listUniqueCardID),int(len(listUniqueCardID)/500))
listThreads = []
intCount = 0
gl_ActualUserData = pd.DataFrame()

for i in range(len(listParts)):

    if listParts[i] == 0 :
        break
    print(f'Thread {i} from {intCount} to {intCount+listParts[i]} is start - {listParts[i]}')
    listThreads.append( threading.Thread( name=f'Thread_{i}', target=th_get_ActualUserData, args=(gl_ActualUserData, listUniqueCardID[ intCount: intCount+listParts[i] ],i , ) ))
    intCount += listParts[i]+1
    listThreads[i].start()

Thread 0 from 0 to 502 is start - 502
0
Thread 1 from 503 to 1005 is start - 502
0
Thread 2 from 1006 to 1508 is start - 502
0
Thread 3 from 1509 to 2011 is start - 502
0
Thread 4 from 2012 to 2514 is start - 502
0
Thread 5 from 2515 to 3017 is start - 502
0
Thread 6 from 3018 to 3520 is start - 502
0
Thread 7 from 3521 to 4023 is start - 502
0
Thread 8 from 4024 to 4526 is start - 502
0
Thread 9 from 4527 to 5029 is start - 502
0
Thread 10 from 5030 to 5532 is start - 502
0
Thread 11 from 5533 to 6035 is start - 502
0
Thread 12 from 6036 to 6538 is start - 502
0
Thread 13 from 6539 to 7041 is start - 502
0Thread 14 from 7042 to 7544 is start - 502

0
Thread 15 from 7545 to 8047 is start - 502
0
Thread 16 from 8048 to 8550 is start - 502
0
Thread 17 from 8551 to 9053 is start - 502
0
Thread 18 from 9054 to 9556 is start - 502
0Thread 19 from 9557 to 10059 is start - 502

0
Thread 20 from 10060 to 10562 is start - 502
0
Thread 21 from 10563 to 11065 is start - 502
0
Thread 22 from 11066

In [13]:
gl_ActualUserData = pd.DataFrame()
count = 0

for i in range(len(listParts)):
    gl_ActualUserData = pd.concat([gl_ActualUserData, pd.read_csv(f'Source_files/JSON/New_CSV_{i}.csv').drop_duplicates()]) 
    count += pd.read_csv(f'Source_files/JSON/New_CSV_{i}.csv').shape[0]
gl_ActualUserData.shape

100100

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100100
100
100

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
200200

200
200
200
200
200
200
200
200
200
200
200
200
200200

200
200
200
200200

200
200
200
200
200200

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
300
300300

300
300
300
300
300
300
300
300
300
300
300300
300

300
300
300300

300
300
300300

300
300
300
300
300
300
300
300
300
300
300
300
300


(16313, 20)

300
300


In [15]:
gl_ActualUserData.drop(['Unnamed: 0'], axis=1)

(214, 19)
Thread 26 is done !

Thread 19 is done !
500
500
500
500


500
(156, 19)
Thread 14 is done !
500
500
(155, 19)
Thread 4 is done !
(218, 19)
Thread 24 is done !
(174, 19)(163, 19)
Thread 9 is done !

Thread 15 is done !
(210, 19)
(245, 19)
Thread 29 is done !
Thread 36 is done !
(256, 19)
Thread 30 is done !
500
(181, 19)

,client_id,gender,birth_date,create_date,nonresident_flag,city,contract_sum,product_category_name,card_id,card_type_name,start_date,fact_close_date,purchase_sum,purchase_count,current_balance_avg_sum,current_balance_sum,current_debit_turn_sum,current_credit_turn_sum,card_type
0,fe60b594364f9f636266ed1ef4f89c32,Ж,1985,2020-01-29,R,Гусь-Хрустальный,0,Договор на текущий счет для дебетовой карты,a83cbc33f164deae41b3339141923cc7,Visa Platinum Rewards,2019,2021,16600,71,39700,25700,220600,201000,dc
1,3012cabca5885ed53d348d6e57dab5de,М,1951,2009-09-24,R,Ишимбай,0,Договор на текущий счет для дебетовой карты,8dae17a830bb05d151034df618efbcad,VISA Classic,2019,-1,4900,2,38000,26800,56300,31300,dc
2,d871ef96820b6c7a1ada8e01a772724e,Ж,1982,2006-12-25,R,Юрга,0,Договор на текущий счет для дебетовой карты,429ca8c43cdc5f3d8c9efc9069ee3fa6,NaN,-1,-1,0,0,0,0,0,0,dc
3,0bacbc2abd314f9fac031d249e485a7b,М,1955,2011-04-15,R,Томск,0,Договор на текущий счет для дебетовой карты,e7f4adb9187b867ebea15503298437a3,Visa Rewards,2021,-1,47800,12,0,0,51500,51500,cc
4,0bacbc2abd314f9fac031d249e485a7b,М,1955,2011-04-15,R,Томск,0,Договор на текущий счет для дебетовой карты,e7d13882154e965dca7070eb8d626f77,Eurocard/MasterCard World,2018,2022,47800,12,0,0,51500,51500,cc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,9acb18399b830419bac9653b52754646,М,1986,2015-05-20,R,Уфа,0,Договор на текущий счет для дебетовой карты,d090cb20c22e2643ec366daf8d26d646,NaN,-1,-1,0,0,0,0,0,0,dc
326,8b3206ca8d77aa2446824971f47c531b,Ж,1996,2022-04-28,R,Казань,30000,Кредитная карта,5a0041efbd5832765caf8b9f861bffa3,NaN,-1,-1,22700,7,0,0,22700,22700,dc
327,d69d3201104ab393c0638839a7284ad7,М,1953,2018-08-08,R,Пятигорск,200000,Кредитная карта,a78c73e84e893179de17071583cf5084,NaN,-1,-1,0,0,0,300,9900,10100,dc
328,a936fb30e2268d2874cdcaa221032915,М,1997,2022-05-12,R,Кемерово,15000,Кредитная карта,e3304eaf55891a175b6abf8030b99da1,NaN,-1,-1,15000,33,0,0,15000,15000,dc


(240, 19)

Thread 1 is done !
Thread 35 is done !500
500

500
500


In [16]:
gl_ActualUserData.to_csv('Source_files/RTG_CSV.csv')

500


(191, 19)500
Thread 40 is done !
500

500(222, 19)
Thread 27 is done !

(205, 19)(202, 19)
Thread 42 is done !

Thread 43 is done !
500
(184, 19)(220, 19)
Thread 12 is done !

Thread 11 is done !
500
(198, 19)
Thread 25 is done !
500
(171, 19)
Thread 7 is done !
500
(316, 19)
Thread 32 is done !
500
(182, 19)
Thread 3 is done !
(236, 19)
Thread 41 is done !
500
(350, 19)
Thread 33 is done !
500
(162, 19)
Thread 13 is done !
500
(285, 19)
Thread 31 is done !
500
(147, 19)500

Thread 16 is done !
500
(206, 19)
Thread 38 is done !
(178, 19)
Thread 2 is done !
(472, 19)
Thread 34 is done !
(209, 19)(208, 19)
Thread 39 is done !

Thread 23 is done !
